In [14]:
import base64


In [15]:
def encode_image(image):
        if type(image) == str:
            with open(image, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')
        else:
            temp_image_path = "./temp.png"
            image.save(temp_image_path)
            return encode_image(temp_image_path)

In [9]:

input_file = "./prompts/command/system_prompt"
output_file = "./prompts/command/image_path"

with open(input_file, "r") as file:
    lines = file.readlines()

extracted_lines = []
for line in lines:
    start_index = line.find("[")
    end_index = line.find("]")
    if start_index != -1 and end_index != -1:
        extracted_lines.append(line[start_index+1:end_index])

with open(output_file, "a") as file:
    for line in extracted_lines:
        file.write(line + ": \n")

In [27]:
def process_file_content(file_path, mapping_file_path):
    # Step 1: Read and process the first file
    with open(file_path, 'r') as file:
        content = file.read()

    result = []
    temp_str = ""
    inside_brackets = False

    for char in content:
        if char == "[":
            if temp_str:
                result.append(temp_str)
                temp_str = ""
            temp_str += char
            inside_brackets = True
        elif char == "]" and inside_brackets:
            temp_str += char
            result.append(temp_str)
            temp_str = ""
            inside_brackets = False
        else:
            temp_str += char

    if temp_str:
        result.append(temp_str)
        

    # Step 2: Read the mapping file and create a dictionary
    mapping = {}
    with open(mapping_file_path, 'r') as file:
        for line in file:
            key, path = line.strip().split(": ")
            mapping[key] = path
    

    # Step 3: Replace elements in the list
    for i, element in enumerate(result):
        if element.startswith("[") and element.endswith("]"):
            
            key = element.strip("[]")
            if key in mapping:
                result[i] = encode_image(mapping[key])
    return result

# Example usage
file_path = input_file
mapping_file_path = output_file
output_list = process_file_content(file_path, mapping_file_path)
